In [44]:
import numpy as np
import matplotlib.pyplot as plt
from math import sqrt
from scipy import stats
from sklearn.metrics import adjusted_rand_score
import pandas as pd
from itertools import product

In [51]:
p_s = [100,200,400]
d_s = [2,5,8]
M_s = [4]
n_percluster_s = [50]
conc_dir_s = [0.1,0.5,1]
rho_s = [5, 10, 20]

# Outer cycle for reading the different datasets and perform the estimation
for p,M,npc in product(p_s, M_s, n_percluster_s):
    # for each of the combination of settings I create one dataframe
    
    list_compare = []
    ar_ind = []
    df_final = []
    
    for dtrue in d_s:

        for conc_dir in conc_dir_s:

            with open("lamb_out/lamb_p_{0}_d_{1}_M_{2}_npc_{3}_out/conc_{4}_out1/df_performance_lamb.csv".format(p,dtrue,M,npc,conc_dir), newline='') as my_csv:
                lamb_df = pd.read_csv(my_csv)


            list_compare.append(lamb_df[["mode_nclus","avg_nclus","ari_best_clus","CI_aris"]])


        for rho in rho_s:

            with open("applam/app_p_{0}_d_{1}_M_{2}_npc_{3}_out/rho_{4}_out_0/df_performance.csv".format(p,dtrue,M,npc,rho), newline='') as my_csv:
                applam_df = pd.read_csv(my_csv)

            list_compare.append(applam_df[["mode_nclus","avg_nclus","ari_best_clus","CI_aris"]])


        df_final = pd.concat(list_compare)

        ar_lamb = [np.repeat(dtrue,3),np.repeat("Lamb",3),conc_dir_s]
        ar_lamb = list(zip(*ar_lamb))

        ar_app = [np.repeat(dtrue,3),np.repeat("APPLAM",3),rho_s]
        ar_app = list(zip(*ar_app))

        ar_ind = [*ar_ind, *ar_lamb,*ar_app]
        
    
    index = pd.MultiIndex.from_tuples(ar_ind, names=["Latent dim", "Model", "Parameter"])
        
    df_final = df_final.set_index(index)
    
    #df_final.to_csv("summarizing_dataframe_p_{0}.csv".format(p))


# Display summarizing dataframes

In [142]:
for p in [100,200,400]:
    with open("summarizing_dataframe_p_{0}.csv".format(p), newline='') as my_csv:
        df = pd.read_csv(my_csv, index_col=[0,1,2])
    
    df["avg_nclus"] = df["avg_nclus"].round(2)
    df["ari_best_clus"] = df["ari_best_clus"].round(2)
    colu = list(map(lambda x: list(map(float,x.strip('()').split(','))),df["CI_aris"]))
    df["CI_aris"] = list(map(lambda x: np.array([1.,1.]) if np.isnan(x).any() else np.around(np.array(x),3), colu))
    
    print("p = ",p)
    
    print(df.to_latex())

p =  100
\begin{tabular}{lllrrrl}
\toprule
  &        &      &  mode\_nclus &  avg\_nclus &  ari\_best\_clus &         CI\_aris \\
Latent dim & Model & Parameter &             &            &                &                 \\
\midrule
2 & Lamb & 0.1  &           5 &       4.76 &           0.94 &  [0.953, 0.955] \\
  &        & 0.5  &           5 &       5.26 &           0.94 &  [0.932, 0.934] \\
  &        & 1.0  &          28 &      27.61 &           0.22 &   [0.268, 0.27] \\
  & APPLAM & 5.0  &           6 &       6.64 &           0.83 &  [0.824, 0.829] \\
  &        & 10.0 &           9 &       9.02 &           0.72 &  [0.717, 0.725] \\
  &        & 20.0 &           6 &       6.07 &           0.91 &  [0.883, 0.889] \\
5 & Lamb & 0.1  &          54 &      54.27 &           0.09 &    [0.099, 0.1] \\
  &        & 0.5  &          62 &      62.65 &           0.08 &  [0.081, 0.081] \\
  &        & 1.0  &          41 &      39.70 &           0.34 &  [0.354, 0.355] \\
  & APPLAM & 5.0  &  